In [1]:
print("Hi! from Agents")

Hi! from Agents


In [2]:
from file_conversion import FileConverter
from crewai import Agent,Crew,Task, LLM
from crewai_tools import FileReadTool,ScrapeWebsiteTool,MDXSearchTool,SerperDevTool
import os
from dotenv import load_dotenv

c:\Users\amish\anaconda3\envs\multi_agent\lib\site-packages\pydantic\fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [3]:
converter_instance  = FileConverter()
converter_instance.pdf_to_md(r"C:\Users\amish\Downloads\RESUME.pdf")

Conversion done! See old_resume.md


In [4]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./old_resume.md')
search_resume = MDXSearchTool(mdx='./old_resume.md')

c:\Users\amish\anaconda3\envs\multi_agent\lib\site-packages\alembic\config.py:577: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(


In [6]:
# 1st Agent: Researcher
first_agent_llm = LLM(
    model=os.getenv("GEMINI_MODEL_NAME"),
    temperature=0.7,
)

researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    ),
    llm=first_agent_llm
)

# 2nd Agent: Profiler(OpenAI LLM)
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    ),
    llm = first_agent_llm
)

# 3rd Agent: Resume Strategist

resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    ),
    llm=first_agent_llm,
)

# 4th Agent: Interview Preparer

interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    ),
    llm = first_agent_llm
)

In [7]:
# 1st Agent Task: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

# 2nd Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

# 3rd Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="new_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

# 4th Agent: Design Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

In [8]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

In [ ]:
job_application_inputs = {
    'job_posting_url': 'https://search.jobs.barclays/job/-/-/13015/81911683408?src=JB-12860',
    'github_url': 'https://github.com/amishkr22',
    'personal_writeup': """I am Amish Nayar, a passionate and driven Computer Science and Engineering undergraduate with a strong
                           focus on Machine Learning, Natural Language Processing, and Large Language Models. My hands-on
                           experience ranges from developing sophisticated AI-powered chatbots and RAG pipelines to fine-tuning
                           cutting-edge models like LLaMA and GEMMA."""
}

In [10]:
result = job_application_crew.kickoff(inputs=job_application_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c0bdc42a-58c3-43ef-95bc-c339b7981ba6                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\amish\anaconda3\envs\multi_agent\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Tech Job Researcher                                                                                     │
│                                                                                                                 │
│  Task: Analyze the job posting URL provided                                                                     │
│  (https://careers.capgemini.com/job/Pune-GenAI-Engineer/1198919201/?feedId=388933&utm_source=LinkedInJobPostin  │
│  gs) to extract key skills, experiences, and qualifications required. Use the tools to gather content and       │
│  identify and categorize the requirements.                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Personal Profiler for Engineers                                                                         │
│                                                                                                                 │
│  Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/amishkr22)     │
│  URLs, and personal write-up (I am Amish Nayar, a passionate and driven Computer Science and Engineering        │
│  undergraduate with a strong                                                                                    │
│                             focus on Machine Learning, Natural Language Processing, and Large Language Models.  │
│  My hands-on                                                                                                    │
│                             experience ranges from developing sophisticated AI-powered chatbots and RAG         │
│  pipelines to fine-tuning                                                                                       │
│                             cutting-edge models like LLaMA and GEMMA.). Utilize tools to extract and            │
│  synthesize information from these sources.                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\amish\anaconda3\envs\multi_agent\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"file_path\": \"./old_resume.md\"}"                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Amish Nayar                                                                                                    │
│  (cid:131) +91-9204971719 # amishkumar1028@gmail.com (cid:239) Amish Nayar § amishkr22                          │
│                                                                                                                 │
│  ## Technical Skills                                                                                            │
│                                                                                                                 │
│  Programming and Development: Python, SQL, Pandas, NumPy, TensorFlow, PyTorch                                   │
│  Machine Learning and AI Tools: ML algorithms, Hugging Face, LangChain, Transformers, LlamaIndex,               │
│  Large Language Models, OpenAI, RAG Framework, Fine-Tuning Methods, MCP(Model Context Protocol),                │
│  Ollama, Llama cpp                                                                                              │
│  Data Visualization and Analysis: Power BI, Matplotlib, Seaborn                                                 │
│  Databases and Storage: MySQL, PostgreSQL, Pinecone, ChromaDB                                                   │
│  Natural Language Processing: NLTK, GPT-4, BERT, SpaCy, Word2Vec, TextBlob                                      │
│  Cloud and Version Control: AWS EC2, AWS Sagemaker, Git                                                         │
│                                                                                                                 │
│  ## Projects                                                                                                    │
│                                                                                                                 │
│  LLAMEDI: Medical Chatbot                                                                                       │
│  - Engineered a chatbot using the LLaMA model from Hugging Face and Pinecone, processing over 5,000             │
│  medical documents to provide real-time responses to healthcare queries.                                        │
│  - Improved query resolution accuracy by 40% through efficient document chunking and indexing using             │
│  PyPDFLoader and RecursiveCharacterTextSplitter.                                                                │
│  - Integrated vector search for rapid information retrieval, reducing response time by 25% compared to          │
│  traditional methods.                                                                                           │
│  MCQGen: Automated Question Generation                                                                          │
│  - Designed an automation pipeline to generate 500+ MCQs with 85% accuracy, decreasing manual effort by         │
│  50% using OpenAI’s GPT-3.5 and LangChain.                                                                      │
│  - Streamlined preprocessing and validation processes, cutting pipeline setup time by 30%.                      │
│  - Enhanced question quality by implementing context-aware prompts, ensuring relevance and correctness for      │
│  diverse educational topics.                                                                                    │
│  RAG Pipeline Chatbot                                                                                           │
│  - Developed a chatbot combining RAG (Retrieval-Augmented Generation) with GPT-4, indexing over                 │
│  10,000 records to provide accurate, context-sensitive responses.                                               │
│  - Achieved a 35% improvement in query resolution accu

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Engineering Interview Preparer                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```                                                                                                            │
│  ## Interview Questions and Talking Points for Amish Nayar                                                      │
│                                                                                                                 │
│  **I. General Questions & Resume Overview:**                                                                    │
│                                                                                                                 │
│  1.  **Tell me about yourself and your background in AI/ML.** (Talking Point: Use this as an opportunity to     │
│  highlight your passion, your focus on Machine Learning, NLP, and LLMs, and your data-driven approach.)         │
│  2.  **Walk me through your resume, highlighting the projects and experiences most relevant to this role.**     │
│  (Talking Point: Focus on projects demonstrating NLP, RAG, LLM fine-tuning, and AI model implementation         │
│  skills. Quantify your achievements whenever possible.)                                                         │
│  3.  **What are your key strengths related to AI/ML?** (Talking Point: Mention your skills in areas like deep   │
│  learning, NLP, LLMs, and AI frameworks. Provide specific examples from your projects.)                         │
│  4.  **What are you most passionate about in the field of AI/ML?** (Talking Point: Connect your interests       │
│  (Generative AI, LLMOps, etc.) to the potential contributions you can make to the company.)                     │
│                                                                                                                 │
│  **II. Project-Specific Questions:**                                                                            │
│                                                                                                                 │
│  *   **LLAMEDI: Medical Chatbot**                                                                               │
│      *   **Question:** Can you elaborate on the challenges you faced while building the LLAMEDI chatbot and     │
│  how you overcame them? (Talking Point: Discuss challenges related to processing large medical datasets,        │
│  ensuring accuracy, and optimizing response time. Highlight your solutions, such as efficient document          │
│  chunking and vector search integration.)                                                                       │
│      *   **Question:** What specific techniques did you use for document chunking and indexing in the LLAMEDI   │
│  project, and why did you choose them? (Talking Point: Discuss the use of PyPDFLoader and                       │
│  RecursiveCharacterTextSplitter, explaining their benefits for handling medical documents.)                     │
│      *   **Question:** How did you evaluate the performance of the LLAMEDI chatbot, and what metrics did you    │
│  use? (Talking Point: Discuss metrics like query resolution accuracy and response time. Explain how you         │
│  measured and improved these metrics.)                                                                          │
│                                                                                                                 │
│  *   **MCQGen: Automated Question Generation**         

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: fb668cea-e063-4bed-83f1-ecef0e7234b5                                                                     │
│  Agent: Engineering Interview Preparer                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c0bdc42a-58c3-43ef-95bc-c339b7981ba6                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```                                                                                              │
│  ## Interview Questions and Talking Points for Amish Nayar                                                      │
│                                                                                                                 │
│  **I. General Questions & Resume Overview:**                                                                    │
│                                                                                                                 │
│  1.  **Tell me about yourself and your background in AI/ML.** (Talking Point: Use this as an opportunity to     │
│  highlight your passion, your focus on Machine Learning, NLP, and LLMs, and your data-driven approach.)         │
│  2.  **Walk me through your resume, highlighting the projects and experiences most relevant to this role.**     │
│  (Talking Point: Focus on projects demonstrating NLP, RAG, LLM fine-tuning, and AI model implementation         │
│  skills. Quantify your achievements whenever possible.)                                                         │
│  3.  **What are your key strengths related to AI/ML?** (Talking Point: Mention your skills in areas like deep   │
│  learning, NLP, LLMs, and AI frameworks. Provide specific examples from your projects.)                         │
│  4.  **What are you most passionate about in the field of AI/ML?** (Talking Point: Connect your interests       │
│  (Generative AI, LLMOps, etc.) to the potential contributions you can make to the company.)                     │
│                                                                                                                 │
│  **II. Project-Specific Questions:**                                                                            │
│                                                                                                                 │
│  *   **LLAMEDI: Medical Chatbot**                                                                               │
│      *   **Question:** Can you elaborate on the challenges you faced while building the LLAMEDI chatbot and     │
│  how you overcame them? (Talking Point: Discuss challenges related to processing large medical datasets,        │
│  ensuring accuracy, and optimizing response time. Highlight your solutions, such as efficient document          │
│  chunking and vector search integration.)                                                                       │
│      *   **Question:** What specific techniques did you use for document chunking and indexing in the LLAMEDI   │
│  project, and why did you choose them? (Talking Point: Discuss the use of PyPDFLoader and                       │
│  RecursiveCharacterTextSplitter, explaining their benefits for handling medical documents.)                     │
│      *   **Question:** How did you evaluate the performance of the LLAMEDI chatbot, and what metrics did you    │
│  use? (Talking Point: Discuss metrics like query resolution accuracy and response time. Explain how you         │
│  measured and improved these metrics.)                                                                          │
│                                                       

In [11]:
print("Done")

Done
